In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from pywander.pathlib import to_absolute_path
from dateutil.parser import parse

In [2]:
file_list = [f'Metacritic_{i}.mhtml' for i in range(1, 86)]

In [3]:
driver = webdriver.Chrome()

result = []

def process_one_file(file_name):
    file_path = f'data/{file_name}'
    file_path = to_absolute_path(file_path)

    print(f'start process file: {file_path}')


    driver.get(file_path)
    page_source = driver.page_source

    soup = BeautifulSoup(page_source, 'html5lib')

    el_item = soup.find_all(name='div', attrs={'data-testid':"filter-results"})

    for e_item in el_item:
        link = e_item.find('a').get('href')
        
        try:
            cover = e_item.find('img').get('src')
        except AttributeError as e:
            cover = ''

        e_title = e_item.find('h3', class_='c-finderProductCard_titleHeading')

        e_order = e_title.find('span')
        e_title_span = e_order.find_next_sibling('span')

        order = e_order.get_text()
        title = e_title_span.get_text()

        order = order.rstrip('.')

        order = order.strip()
        title = title.strip()

        e_date = e_item.find('div', class_='c-finderProductCard_meta').find('span')
        
        date = e_date.get_text().strip()


        dt_date = parse(date)

        e_score = e_item.find('div', class_='c-siteReviewScore').find('span')
        score = e_score.get_text().strip()
        
        order = order.replace(',', '')
        order = order.replace(' ', '')
        
        item = {
            'title': title,
            'order': int(order),
            'cover': cover,
            'link': link,
            'date': dt_date,
            'score': int(score)
        }

        result.append(item)



In [4]:
for file_name in file_list:
    process_one_file(file_name)


start process file: d:\github\metacritic_game\data\Metacritic_1.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_2.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_3.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_4.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_5.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_6.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_7.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_8.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_9.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_10.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_11.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_12.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_13.mhtml
start process file: d:\github\metacritic_game\data\Metacritic_14.mhtml
start process f

In [5]:
result

[{'title': 'Disco Elysium: The Final Cut',
  'order': 1,
  'cover': 'https://www.metacritic.com/a/img/resize/a1cfbe720f0b33b47fdd5bd5b7a790aba92a807f/catalog/provider/6/12/6-1-779717-52.jpg?auto=webp&fit=cover&height=72&width=48',
  'link': 'https://www.metacritic.com/game/disco-elysium-the-final-cut/',
  'date': datetime.datetime(2021, 3, 30, 0, 0),
  'score': 97},
 {'title': 'Grand Theft Auto V',
  'order': 2,
  'cover': 'https://www.metacritic.com/a/img/resize/3fd5452825d50a184cae81892840c3b6255aa897/catalog/provider/6/3/6-1-402391-13.jpg?auto=webp&fit=cover&height=72&width=48',
  'link': 'https://www.metacritic.com/game/grand-theft-auto-v/',
  'date': datetime.datetime(2015, 4, 13, 0, 0),
  'score': 96},
 {'title': "Baldur's Gate 3",
  'order': 3,
  'cover': 'https://www.metacritic.com/a/img/resize/28be45b25d1f22e0dbe65849f2f61ca0f32f651c/catalog/provider/7/2/7-1695438055.jpg?auto=webp&fit=cover&height=72&width=48',
  'link': 'https://www.metacritic.com/game/baldurs-gate-3/',
  'da

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(result)

In [ ]:
df

In [9]:
df.columns

Index(['title', 'order', 'cover', 'link', 'date', 'score'], dtype='object')

In [12]:
import xlwings as xw
book = xw.Book()

In [13]:
sheet1 = book.sheets[0]
sheet1["A1"].options(index=False).value = df


In [14]:
book.save('metacritic_game.xlsx')

In [15]:
driver.quit()
